### Phase 4: Build machine learning models for forecast

In [1]:
# measure execution time
%load_ext autotime

time: 0 ns (started: 2023-07-03 19:54:49 +03:00)


In [2]:
# standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

time: 2.7 s (started: 2023-07-03 19:54:49 +03:00)


### Load the dataset

In [3]:
# read dataset
time_series = pd.read_csv('C:/Users/SK/Desktop/Diploma_Thesis/Datasets/San-Francisco-Yellow-Cabs/Files/time_series_per15Minutes.txt')

# drop unecessary columns
time_series.drop(["Traj ID","Taxi ID"],axis=1,inplace=True)

# print first two rows
time_series.shape

(1000, 674)

time: 156 ms (started: 2023-07-03 19:54:52 +03:00)


In [4]:
time_series.head(5)

,Path,Length,"(Timestamp('2008-05-18 00:00:00'), Timestamp('2008-05-18 00:15:00'))","(Timestamp('2008-05-18 00:15:00'), Timestamp('2008-05-18 00:30:00'))","(Timestamp('2008-05-18 00:30:00'), Timestamp('2008-05-18 00:45:00'))","(Timestamp('2008-05-18 00:45:00'), Timestamp('2008-05-18 01:00:00'))","(Timestamp('2008-05-18 01:00:00'), Timestamp('2008-05-18 01:15:00'))","(Timestamp('2008-05-18 01:15:00'), Timestamp('2008-05-18 01:30:00'))","(Timestamp('2008-05-18 01:30:00'), Timestamp('2008-05-18 01:45:00'))","(Timestamp('2008-05-18 01:45:00'), Timestamp('2008-05-18 02:00:00'))",...,"(Timestamp('2008-05-24 21:30:00'), Timestamp('2008-05-24 21:45:00'))","(Timestamp('2008-05-24 21:45:00'), Timestamp('2008-05-24 22:00:00'))","(Timestamp('2008-05-24 22:00:00'), Timestamp('2008-05-24 22:15:00'))","(Timestamp('2008-05-24 22:15:00'), Timestamp('2008-05-24 22:30:00'))","(Timestamp('2008-05-24 22:30:00'), Timestamp('2008-05-24 22:45:00'))","(Timestamp('2008-05-24 22:45:00'), Timestamp('2008-05-24 23:00:00'))","(Timestamp('2008-05-24 23:00:00'), Timestamp('2008-05-24 23:15:00'))","(Timestamp('2008-05-24 23:15:00'), Timestamp('2008-05-24 23:30:00'))","(Timestamp('2008-05-24 23:30:00'), Timestamp('2008-05-24 23:45:00'))","(Timestamp('2008-05-24 23:45:00'), Timestamp('2008-05-25 00:00:00'))"
0,"[254758131, 254758128, 254758128, 27400639]",4,1,0,3,1,0,2,1,2,...,2,1,1,2,0,0,1,2,0,1
1,"[1126161509, 254466686, 254466686, 254466686, ...",15,1,1,0,0,3,2,2,1,...,0,1,0,0,0,1,1,0,0,0
2,"[514412999, 85562484]",2,1,0,2,0,1,0,2,1,...,2,0,0,0,0,0,0,0,2,0
3,"[397125833, 397125832, 397257567, 283535238, 3...",11,2,0,0,0,0,0,0,0,...,0,0,1,0,2,0,0,0,1,0
4,"[682524826, 112926340]",2,1,0,0,2,0,2,1,0,...,0,0,0,0,0,0,0,0,0,2


time: 47 ms (started: 2023-07-03 19:54:52 +03:00)


In [5]:
# this list contains the column names
columns = ["Path","Length"]

# generate the columns of the dataset
i =2
while(True):
    if i == 2:
        columns.append(pd.to_datetime('2008-05-18 00:00:00'))
    else:
        columns.append(columns[i-1] + timedelta(seconds=900))
    
    if (columns[i]>=pd.to_datetime('2008-05-24 23:59:59.000130')):
        break
    
    i+=1

# delete the last timestamp
columns.pop()

# assign new column names to our dataframe
time_series.columns = columns

# print first two rows
time_series.head(2)

,Path,Length,2008-05-18 00:00:00,2008-05-18 00:15:00,2008-05-18 00:30:00,2008-05-18 00:45:00,2008-05-18 01:00:00,2008-05-18 01:15:00,2008-05-18 01:30:00,2008-05-18 01:45:00,...,2008-05-24 21:30:00,2008-05-24 21:45:00,2008-05-24 22:00:00,2008-05-24 22:15:00,2008-05-24 22:30:00,2008-05-24 22:45:00,2008-05-24 23:00:00,2008-05-24 23:15:00,2008-05-24 23:30:00,2008-05-24 23:45:00
0,"[254758131, 254758128, 254758128, 27400639]",4,1,0,3,1,0,2,1,2,...,2,1,1,2,0,0,1,2,0,1
1,"[1126161509, 254466686, 254466686, 254466686, ...",15,1,1,0,0,3,2,2,1,...,0,1,0,0,0,1,1,0,0,0


time: 125 ms (started: 2023-07-03 19:54:57 +03:00)


### Transform the dataset

In [ ]:
# reshape the data from column format to row format
id_cols = ['Path', 'Length']
time_cols = time_series.iloc[:,2:].columns
time_series = time_series.melt(id_vars=id_cols, value_vars=time_cols, var_name='Time Column', value_name='Traffic Flow')
time_series['Time Column'] = pd.to_datetime(time_series['Time Column'])

# sort rows by Path and Time
time_series.sort_values(by=['Path','Time Column'],inplace=True)

# print first rows of the dataset
time_series.head(5)

In [ ]:
# convert path column from categorical to numerical
from sklearn.preprocessing import LabelEncoder

# create an instance of LabelEncoder
label_encoder = LabelEncoder()

# Fit label encoder on the 'Path' column
time_series['Path'] = label_encoder.fit_transform(time_series['Path'])

# sort the data based in Path column
time_series = time_series.sort_values(by=['Path','Time Column'])

In [ ]:
time_series

### Convert the time series dataset to a supervised problem

In [ ]:
'''
from pandas import DataFrame
from pandas import concat

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	"""
	Frame a time series as a supervised learning dataset.
	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		n_out: Number of observations as output (y).
		dropnan: Boolean whether or not to drop rows with NaN values.
	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('Traffic Flow %d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('Traffic Flow %d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('Traffic Flow %d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

data = pd.DataFrame()

for item in time_series['Path'].unique():
    x = series_to_supervised(time_series[time_series['Path'] == item].drop(['Length','Path'],axis=1), 16,1)
    x['Path'] = item
    x['Length'] = time_series[time_series['Path'] == item]['Length']
    
    data = pd.concat([data,x])
'''

### Feature Creation From Time Information
We define some features based on time series timestamps.

In [ ]:
time_series['hour'] = time_series['Time Column'].dt.hour
time_series['dayofweek'] = time_series['Time Column'].dt.dayofweek
time_series['day'] = time_series['Time Column'].dt.day
time_series['minute'] = time_series['Time Column'].dt.minute

In [ ]:
# Circular Encoding for cyclic time features
time_series['hour_sin'] = np.sin(2 * np.pi * time_series['hour'] / 24)
time_series['hour_cos'] = np.cos(2 * np.pi * time_series['hour'] / 24)

In [ ]:
time_series.head(3)

### Train - Test split

In [ ]:
# define train data
train = time_series.loc[time_series['Time Column'] < '2008-05-24']
train.sort_values(by='Time Column', inplace=True)

# define test data
test = time_series.loc[time_series['Time Column'] >= '2008-05-24']
test.sort_values(by='Time Column', inplace=True)

print("Train set shape: ",train.shape)
print("Test set shape: ",test.shape)

### Define Features and Labels

In [ ]:
# define features
features = ['Path', 'Length', 'hour', 'dayofweek', 'day', 'minute',
            'hour_sin', 'hour_cos']

# define labels
labels = ['Traffic Flow']

# create train set
X_train = train[features]
y_train = train[labels]

# create test set
X_test = test[features]
y_test = test[labels]

### Train a single XGboost model
We will fit all the data in this model!

In [ ]:
import xgboost as xgb

In [ ]:
# Define XGBoost regressor
reg = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=5000,
    early_stopping_rounds=5,
    learning_rate=0.01,
    max_depth=5,
    gamma=0.3,
    alpha = 0.1
)

# Train the XGBoost model with feature weights
reg.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], verbose=True)

### Feature Importance

In [ ]:
# Show how much each of the features was used in our regression analysis
plt.figure(figsize=(15,5))
fi = pd.DataFrame(data=reg.feature_importances_,index=reg.feature_names_in_,columns=['importance'])
fi.sort_values('importance').plot(kind='barh', title='Feature Importance')
plt.show()

### Forecast on test set

In [ ]:
# Timeseries forecating on test set
y_pred = reg.predict(X_test)

### Evaluate the results

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Calculate the MAE
mae = mean_absolute_error(y_test, y_pred)
print("Mae is: ",mean_absolute_error(y_test, y_pred))

# Calculate RMSE score
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("RMSE score:", rmse)

In [ ]:
y_test['pred'] = y_pred
y_test['p'] = X_test['Path']

In [ ]:
y_test[y_test['p']==23].plot()

### Train multiple XGboost models
Fit only the data of one path in each model

In [ ]:
# Define dataframe to save the scores
scores = pd.DataFrame(columns=['MAE','RMSE'],index=time_series.Path.unique())

In [ ]:
regressors = {}  # Dictionary to store regressor objects

for item in X_train['Path'].unique():
    
    sample = time_series[time_series['Path'] == item]
    
    # define train data
    s_train = sample.loc[sample['Time Column'] < '2008-05-24']
    s_train.sort_values(by='Time Column', inplace=True)

    # define test data
    s_test = sample.loc[sample['Time Column'] >= '2008-05-24']
    s_test.sort_values(by='Time Column', inplace=True)

    # define features
    s_features = ['Path', 'Length', 'hour', 'dayofweek', 'day', 'minute',
                'hour_sin', 'hour_cos','rolling_mean','rolling_std']

    # define labels
    s_labels = ['Traffic Flow']

    # create train set
    s_X_train = s_train[s_features]
    s_y_train = s_train[s_labels]

    # create test set
    s_X_test = s_test[s_features]
    s_y_test = s_test[s_labels]

    regressor_name = f"regressor_{item}"  # Name of the regressor
    
    print(regressor_name)
    
    regressors[regressor_name] = xgb.XGBRegressor(
        objective='reg:squarederror',
        n_estimators=5000,
        early_stopping_rounds=5,
        learning_rate=0.01,
        max_depth=3,
        colsample_bytree=0.8,
        gamma=0.3
    )
    
    # Train the XGBoost model with feature weights
    reg.fit(s_X_train, s_y_train, eval_set=[(s_X_train,s_y_train), (s_X_test,s_y_test)], verbose=False)
    
    # Timeseries forecating on test set
    y_pred = reg.predict(s_X_test)
    
    # Calculate the MAE
    mae = mean_absolute_error(s_y_test, y_pred)

    # Calculate RMSE score
    rmse = mean_squared_error(s_y_test, y_pred, squared=False)
    
    scores.at[item,'MAE'] = mae
    scores.at[item,'RMSE'] = rmse

In [ ]:
scores